We compute the variance of standard Gaussian

$$E[X^2] = \int x^2 p(x) dx, \quad p(x) = \mathcal{N}(0,1)$$

Analytical solution gives $E[X^2] = 1.$

In [9]:
import numpy as np

import torch
from gpytorch.likelihoods import GaussianLikelihood
from gpytorch.kernels import ScaleKernel, RBFKernel
from bqtorch.models.gp_regression import GP
from bqtorch.methods.base import VanillaBQGaussPrior


Define the function

In [3]:
def func(x):
    return x.pow(2)

Create data where $x$ is sampled from Gaussian distribution

In [4]:
N = 10
x = torch.randn(N, 1)
y = func(x).squeeze()

Create GP model

In [5]:
rbf = ScaleKernel(RBFKernel())
likelihood = GaussianLikelihood()
likelihood.noise = 0.01
gp = GP(x, y, rbf, likelihood)

Prior $p(x)$

In [7]:
prior_mean = torch.from_numpy(np.array([[0.]], dtype=np.float32))
prior_variance = 1.


Create the quadrature

In [10]:
bq = VanillaBQGaussPrior(gp, prior_mean, prior_variance)
bq.optimize(lr=0.01, epoch=100, verbose=False)
integral = bq.integrate()

The output of Bayesian quadrature

In [12]:
integral[0].item()

0.9225306510925293

The output of MCMC estimate

In [13]:
y.mean().item()

0.7651674151420593